In [102]:
import json
import scipy as sp
import time
import numpy as np
import math

import hnswlib
from annoy import AnnoyIndex
from lshashpy3 import LSHash

In [103]:
TEST_FILE = "10000"

In [104]:
def read_source(size: str = "1000"):
    with open(f"./../.data/example_embeddings_{size}.json", "r") as f:
        source = json.load(f)
    train_split = int(len(source) * 0.9)
    train = source[:train_split]
    test = source[train_split:]
    return train, test

In [105]:
class KDTree:
    def __init__(self, vectors, leaf_size: int = 10) -> None:
        self.vectors = vectors
        self.leaf_size = leaf_size

    def construct(self) -> None:
        self.kdtree = sp.spatial.KDTree(self.vectors, leafsize = self.leaf_size)
        return
    
    def search(self, query, top_n: int = 20) -> list:
        return self.kdtree.query(query, k = top_n)


In [106]:
class HierarchicalNavigableSmallWorld:
    def __init__(self, vectors, vector_size: int = 768, space: str = "l2") -> None:
        self.vectors = np.array(vectors)
        self.hnsw = hnswlib.Index(space, vector_size)
        self.hnsw.init_index(max_elements = 1000000, ef_construction = 200, M = 16)

    def construct(self) -> None:
        self.hnsw.add_items(self.vectors)
        return
    
    def search(self, query, top_n: int = 20) -> list:
        return self.hnsw.knn_query(query, k = top_n)


In [107]:
class Annoy:
    def __init__(self, vectors, no_of_trees: int = 10, dimension_of_input: int = 768, distance_method: str = "euclidean") -> None:
        """
        distance_method can be one of the following:
        "angular", "euclidean", "manhattan", "hamming", "dot"
        (default to be "euclidean")
        """
        self.vectors = vectors
        self.no_of_trees = no_of_trees
        self.ann = AnnoyIndex(dimension_of_input, distance_method)

    def construct(self) -> None:
        for index, vector in enumerate(self.vectors):
            self.ann.add_item(index, vector)
        self.ann.build(self.no_of_trees)
        return

    def search(self, query, top_n: int = 20) -> list:
        nn = self.ann.get_nns_by_vector(query, top_n)
        return nn


In [108]:
class LSH:
    def __init__(self, vectors, no_of_hash_tables: int = 1, no_of_hash_bits: int = 3, dimension_of_input: int = 768) -> None:
        self.vectors = vectors
        self.no_of_hash_tables = no_of_hash_tables
        self.no_of_hash_bits = no_of_hash_bits
        self.dimension_of_input = dimension_of_input

        # Runtime variable
        self.hash_table = LSHash(no_of_hash_bits, dimension_of_input, num_hashtables=no_of_hash_tables)
        
    def construct(self) -> None:
        for vector in self.vectors:
            self.hash_table.index(vector)
        return
    
    def search(self, query, top_n: int = 20) -> list:
        nn = self.hash_table.query(query, num_results = top_n)
        return nn


In [109]:
class NaiveSearch:
    def __init__(self, vectors, distance_method: str = "euclidean") -> None:
        """
        Distance method can be one of the following:
        "angular", "euclidean", "manhattan", "hamming", or "dot"
        """
        self.vectors = vectors
        if distance_method == "euclidean":
            self.distance_method = self._euclidean
        elif distance_method == "angular":
            self.distance_method = self._angular
        elif distance_method == "manhatten":
            self.distance_method = self._manhatten
        elif distance_method == "hamming":
            self.distance_method = self._hamming
        elif distance_method == "dot":
            self.distance_method = self._dot

    def construct(self):
        return

    def search(self, query, top_n: int = 20) -> list:
        distances = []
        for vector in self.vectors:
            distances.append((self.distance_method(query, vector), vector))
        distances = sorted(distances, key= lambda x: x[0])
        return distances[1][:top_n]

    def _euclidean(self, point1, point2) -> float:
        # Convert points to numpy arrays
        point1 = np.array(point1)
        point2 = np.array(point2)
        return np.linalg.norm(point2 - point1)

    def _angular(self, point1, point2) -> float:
        # Convert points to numpy arrays
        point1 = np.array(point1)
        point2 = np.array(point2)

        # Normalize the points
        norm1 = np.linalg.norm(point1)
        norm2 = np.linalg.norm(point2)
        norm_point1 = point1 / norm1
        norm_point2 = point2 / norm2

        # Calculate the dot product
        dot_product = np.dot(norm_point1, norm_point2)

        # Calculate the angle between the points using arccosine
        angle = np.arccos(dot_product)

        # Convert angle from radians to degrees
        angle_deg = np.degrees(angle)

        return angle_deg
            
    def _manhatten(self, point1, point2) -> float:
        distance = sum(abs(x - y) for x, y in zip(point1, point2))
        return distance

    def _hamming(self, point1, point2) -> float:
        distance = sum(x != y for x, y in zip(point1, point2))
        return distance

    def _dot(self, point1, point2) -> float:

        point1 = np.array(point1)
        point2 = np.array(point2)

        distance = np.dot(point1, point2)
        return distance


In [110]:
class KMeansTree:
    def __init__(self, k: int, vectors: np.array, depth: int=0, min_cluster: int=0,  distance_method: str = "dot") -> None:
        self.vectors = np.array(vectors)
        self.k = k
        self.depth = depth
        self.distance_method = distance_method

        # Force stop clustering even if the depth is not reached
        if min_cluster == 0:
            self.min_cluster = 20 * math.log(self.vectors.shape[0])
        else:
            # Skip min_cluster if the min_cluster is smaller than 0
            self.min_cluster = min_cluster

        """
        # Force continuous clustering even if the depth is reached
        if max_cluster == 0:
            self.max_cluster = 40 * math.log(vectors.shape[0])
        else:
            self.max_cluster = max_cluster
        """

        # runtime
        self.root = None

    def construct(self):
        leaf_node = KMeansTreeLeaf(self.k, self.vectors, distance_method=self.distance_method)
        branches = leaf_node.extrusion(self.min_cluster)
        self.root = KMeansTreeBranch(child_branches=branches, distance_method=self.distance_method)
        for i in range(self.depth-1): # Depth minus one because root has one unit of depth
            temp_branches = []
            for branch in branches:
                print(type(branch))
                if isinstance(branch, KMeansTreeBranch):
                    print("Hi")
                    continue
                new_layer_branches: list[KMeansTreeBranch] = branch.child_leaf.extrusion(self.min_cluster)
                branch.child_branches = new_layer_branches
                temp_branches.extend(new_layer_branches)
                branch.child_leaf = None
            branches = [j for j in temp_branches]

    def search(self, vector: np.array) -> list:
        branches = self.root.child_branches
        while(isinstance(branches, KMeansTreeBranch)):
            min_distance = 0x3f3f3f3f
            flag = -1
            for j, branch in enumerate(branches):
                if (distance := branch.distance_to_center_point(vector)) < min_distance:
                    flag = j
                    min_distance = distance
                #print(f"Distance with branch {j}: {distance}")
            #print()
            branches = branches[flag].child_branches
        leaf = branches[0].child_leaf
        return leaf.vectors
    
    def print(self):
        branches = self.root.child_branches
        for i in range(self.depth):
            temp_branches = []
            for j in branches:
                temp_branches.extend(j.child_branches)
            branches = temp_branches.copy()
            print(f"Floor {i}: {len(branches)}")


class KMeansTreeBranch:
    def __init__(self, child_leaf = None, child_branches: list = [], center_point: np.array = None, distance_method: str = "dot") -> None:
        self.child_leaf = child_leaf
        self.child_branches = child_branches
        self.center_point = center_point
        if distance_method == "euclidean":
            self.distance_method = self._euclidean
        elif distance_method == "angular":
            self.distance_method = self._angular
        elif distance_method == "manhatten":
            self.distance_method = self._manhatten
        elif distance_method == "hamming":
            self.distance_method = self._hamming
        elif distance_method == "dot":
            self.distance_method = self._dot

    def distance_to_center_point(self, vector: np.array) -> None:
        return self.distance_method(vector, self.center_point)
    
    def _euclidean(self, point1, point2) -> float:
        # Convert points to numpy arrays
        point1 = np.array(point1)
        point2 = np.array(point2)
        return np.linalg.norm(point2 - point1)

    def _angular(self, point1, point2) -> float:
        # Convert points to numpy arrays
        point1 = np.array(point1)
        point2 = np.array(point2)

        # Normalize the points
        norm1 = np.linalg.norm(point1)
        norm2 = np.linalg.norm(point2)
        norm_point1 = point1 / norm1
        norm_point2 = point2 / norm2

        # Calculate the dot product
        dot_product = np.dot(norm_point1, norm_point2)

        # Calculate the angle between the points using arccosine
        angle = np.arccos(dot_product)

        # Convert angle from radians to degrees
        angle_deg = np.degrees(angle)

        return angle_deg
            
    def _manhatten(self, point1, point2) -> float:
        distance = sum(abs(x - y) for x, y in zip(point1, point2))
        return distance

    def _hamming(self, point1, point2) -> float:
        distance = sum(x != y for x, y in zip(point1, point2))
        return distance

    def _dot(self, point1, point2) -> float:

        point1 = np.array(point1)
        point2 = np.array(point2)

        distance = np.dot(point1, point2)
        return distance


class KMeansTreeLeaf:
    def __init__(self, k: int, vectors: np.array, distance_method: str = "dot") -> None:
        """
        Vectors should be 2-D numpy array
        """
        self.k = k
        self.vectors = vectors
        self.center_point = np.zeros(shape=vectors.shape[1])
        self.distance_method = distance_method
        
        self._temp_vectors = []
        if distance_method == "euclidean":
            self.distance_method = self._euclidean
        elif distance_method == "angular":
            self.distance_method = self._angular
        elif distance_method == "manhatten":
            self.distance_method = self._manhatten
        elif distance_method == "hamming":
            self.distance_method = self._hamming
        elif distance_method == "dot":
            self.distance_method = self._dot
    
    def update_center_point(self) -> bool:
        """
        Return if it is changed
        """
        old_center_point = np.copy(self.center_point)
        self.center_point = np.mean(self.vectors, axis=0)
        isChanged = True
        if np.allclose(old_center_point, self.center_point):
            isChanged = False
        return isChanged
    
    def append_new(self, vector: np.array) -> None:
        self._temp_vectors.append(vector)

    def finish_append(self) -> None:
        self.vectors = np.stack(self._temp_vectors, axis=0)
        self._temp_vectors = []
    
    def distance_to_center_point(self, vector: np.array) -> None:
        return self.distance_method(vector, self.center_point)

    def extrusion(self, min_cluster: int, max_iter: int = 100) -> KMeansTreeBranch:
        """
        Do KNN optimization
        Return KTreeBranch that connected to lower layer KTreeLeaf
        """
        # No extrusion if child is less than something
        if min_cluster < 0:
            # Does not consider minimum clustering size
            pass
        elif self.vectors.shape[0] < min_cluster:
            # If current leaf does not reach the minimum clustering size limit
            temp = KMeansTreeLeaf(self.k, self.vectors, distance_method=self.distance_method)
            temp.update_center_point()
            return [KMeansTreeBranch(child_leaf = temp, center_point=temp.center_point, distance_method=self.distance_method)]

        # Extrusion
        leaves = [KMeansTreeLeaf(self.k,
                            self.vectors[\
                                int(self.vectors.shape[0]*(i)/self.k)\
                                    :int(self.vectors.shape[0]*(i+1)/self.k), :], distance_method=self.distance_method)\
                                        for i in range(self.k)]
        # KNN on the extrusion parts
        for i in range(max_iter):
            isChanged = False
            for j in leaves:
                if j.update_center_point():
                    # If one of the center point changed, continue iteration
                    isChanged = True
            if isChanged == False:
                break
            for j in leaves:
                for vector in j.vectors:
                    min_distance = 0x3f3f3f3f
                    flag = -1
                    for index, leaf in enumerate(leaves):
                        # Compare each vector to center of each node
                        distance = leaf.distance_to_center_point(vector)
                        if distance < min_distance:
                            min_distance = distance
                            flag = index
                    leaves[flag].append_new(vector)
            for j in leaves:
                j.finish_append()
        branches = [KMeansTreeBranch(child_leaf = leaves[i], center_point=leaves[i].center_point, distance_method=self.distance_method) for i in range(self.k)]
        return branches
    
    def _euclidean(self, point1, point2) -> float:
        # Convert points to numpy arrays
        point1 = np.array(point1)
        point2 = np.array(point2)
        return np.linalg.norm(point2 - point1)

    def _angular(self, point1, point2) -> float:
        # Convert points to numpy arrays
        point1 = np.array(point1)
        point2 = np.array(point2)

        # Normalize the points
        norm1 = np.linalg.norm(point1)
        norm2 = np.linalg.norm(point2)
        norm_point1 = point1 / norm1
        norm_point2 = point2 / norm2

        # Calculate the dot product
        dot_product = np.dot(norm_point1, norm_point2)

        # Calculate the angle between the points using arccosine
        angle = np.arccos(dot_product)

        # Convert angle from radians to degrees
        angle_deg = np.degrees(angle)

        return angle_deg
            
    def _manhatten(self, point1, point2) -> float:
        distance = sum(abs(x - y) for x, y in zip(point1, point2))
        return distance

    def _hamming(self, point1, point2) -> float:
        distance = sum(x != y for x, y in zip(point1, point2))
        return distance

    def _dot(self, point1, point2) -> float:

        point1 = np.array(point1)
        point2 = np.array(point2)

        distance = np.dot(point1, point2)
        return distance

In [111]:
def benchmark(SA, name, test):
    test_result = {
            "input_parameters": {
                    "source_file": TEST_FILE,
                    "tested_method": name,
                },
            "index_time": 0, # Note how much time is used for indexing
            "search_time": 0, # Count how many seconds was saved when do the search after indexing
            }
    time_0 = time.time()
    SA.construct()
    time_1 = time.time()
    test_result["index_time"] = time_1 - time_0

    for i in test:
        time_0 = time.time()
        SA.search(i)
        time_1 = time.time()

        test_result["search_time"] += time_1 - time_0

    test_result["search_time"] /= len(test)

    print(test_result)

In [112]:
train, test = read_source()

In [113]:
hnsw = HierarchicalNavigableSmallWorld(train)
kdt = KDTree(train)
ann = Annoy(train)
lsh = LSH(train)
ns = NaiveSearch(train)
kmt = KMeansTree(2, train)

In [114]:
result_kdt = benchmark(kdt, "KDTree", test)
result_lsh = benchmark(lsh, "LSH", test)
result_ann = benchmark(ann, "Annoy", test)
result_hnsw = benchmark(hnsw, "HNSW", test)
result_ns = benchmark(ns, "NaiveSearch", test)


{'input_parameters': {'source_file': '10000', 'tested_method': 'KDTree'}, 'index_time': 0.019246578216552734, 'search_time': 0.00043706417083740236}
{'input_parameters': {'source_file': '10000', 'tested_method': 'LSH'}, 'index_time': 0.024811983108520508, 'search_time': 0.012312817573547363}
{'input_parameters': {'source_file': '10000', 'tested_method': 'Annoy'}, 'index_time': 0.009606122970581055, 'search_time': 5.4521560668945314e-05}
{'input_parameters': {'source_file': '10000', 'tested_method': 'HNSW'}, 'index_time': 0.025594472885131836, 'search_time': 4.805803298950195e-05}
{'input_parameters': {'source_file': '10000', 'tested_method': 'NaiveSearch'}, 'index_time': 1.9073486328125e-06, 'search_time': 0.03263514995574951}


In [115]:
result_kmt = benchmark(kmt, "KDTree", test)


{'input_parameters': {'source_file': '10000', 'tested_method': 'KDTree'}, 'index_time': 0.525092363357544, 'search_time': 2.1457672119140626e-07}


In [117]:
kmt = KMeansTree(2, train, distance_method="euclidean")
benchmark(kmt, "KDTree", test)
kmt = KMeansTree(2, train, distance_method="angular")
benchmark(kmt, "KDTree", test)
kmt = KMeansTree(2, train, distance_method="manhatten")
benchmark(kmt, "KDTree", test)
kmt = KMeansTree(2, train, distance_method="dot")
benchmark(kmt, "KDTree", test)

{'input_parameters': {'source_file': '10000', 'tested_method': 'KDTree'}, 'index_time': 0.07848715782165527, 'search_time': 1.4066696166992187e-07}
{'input_parameters': {'source_file': '10000', 'tested_method': 'KDTree'}, 'index_time': 0.17107510566711426, 'search_time': 1.430511474609375e-07}
{'input_parameters': {'source_file': '10000', 'tested_method': 'KDTree'}, 'index_time': 2.0525853633880615, 'search_time': 1.6927719116210937e-07}
{'input_parameters': {'source_file': '10000', 'tested_method': 'KDTree'}, 'index_time': 0.49854612350463867, 'search_time': 1.5497207641601563e-07}
